<a href="https://colab.research.google.com/github/seanzhu815/402/blob/main/402Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 5 sample code

## Import Modules

In [2]:
import pandas            as pd   
import numpy             as np   
import scipy             as scp  
import matplotlib.pyplot as plt  
import os                as os   
from datetime import date as dd  # for dates
from scipy import optimize
from scipy import stats
import statsmodels.api as sm

# Datafiles

## Part 1

### Q1.1

In [3]:
r = 0.02                          # risk free rate
d =  1-0.0476                         # return in down state
u =  1+0.05                         # return in up state
p =  (1+r-d)/(u-d)                         # risk neutral probability of up state
print(f'The probability is {p :.2f}.')

The probability is 0.69.


### Q 1.2

In [4]:
S0 =   100                                         # Fill in stock prices at all nodes
Su =   105         
Sd =   95.24 
Suu =   110.25
Sud = 100
Sdd = 90.7
K =   95                                          # Strike price
Cuu = max(Suu-K ,0)                                        # Calculate call payoff at all nodes
Cud =  max(Sud-K ,0)
Cdd =  max(Sdd-K ,0)
Cu = (p*Cuu + (1-p)*Cud)/(1+r)                  
Cd = (p*Cud + (1-p)*Cdd)/(1+r)
C = (p*Cu + (1-p)*Cd)/(1+r)
print(f'The option price at node u is {Cu :.2f}.')
print(f'The option price at node d is {Cd :.2f}.')
print(f'The option price at node 0 is {C :.2f}.')

The option price at node u is 11.86.
The option price at node d is 3.40.
The option price at node 0 is 9.08.


### Q 1.3 

In [8]:
H0 = (Cu - Cd)/(u*S0-d*S0)                           # Fill in the hedge ratios
Hu = (Cuu-Cud)/(S0*u**2-S0*u*d)
Hd = (Cud-Cdd)/(S0*u*d-S0*d*d)
print(f'The hedge ratio at node 0 is {H0 :.2f}.')
print(f'The hedge ratio at node u is {Hu :.2f}.')
print(f'The hedge ratio at node d is {Hd :.2f}.')

The hedge ratio at node 0 is 0.87.
The hedge ratio at node u is 1.00.
The hedge ratio at node d is 0.54.


### Q 1.4

In [9]:
S0 =   100                                         # Fill in stock prices at all nodes
Su =   105         
Sd =   95.24 
Suu =   110.25
Sud = 100
Sdd = 90.7
K =   105                                          # Strike price
Cuu = max(Suu-K ,0)                                        # Calculate call payoff at all nodes
Cud =  max(Sud-K ,0)
Cdd =  max(Sdd-K ,0)
Cu = (p*Cuu + (1-p)*Cud)/(1+r)                  
Cd = (p*Cud + (1-p)*Cdd)/(1+r)
C = (p*Cu + (1-p)*Cd)/(1+r)
print(f'The option price at node u is {Cu :.2f}.')
print(f'The option price at node d is {Cd :.2f}.')
print(f'The option price at node 0 is {C :.2f}.')

The option price at node u is 3.56.
The option price at node d is 0.00.
The option price at node 0 is 2.42.


In [10]:
H0 = (Cu - Cd)/(u*S0-d*S0)                           # Fill in the hedge ratios
Hu = (Cuu-Cud)/(S0*u**2-S0*u*d)
Hd = (Cud-Cdd)/(S0*u*d-S0*d*d)
print(f'The hedge ratio at node 0 is {H0 :.2f}.')
print(f'The hedge ratio at node u is {Hu :.2f}.')
print(f'The hedge ratio at node d is {Hd :.2f}.')

The hedge ratio at node 0 is 0.37.
The hedge ratio at node u is 0.51.
The hedge ratio at node d is 0.00.


## Part 2

### Q 2.1

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
datafile1='/content/drive/MyDrive/MGTF402/options_data_2022.xlsx'

In [15]:
options_data = pd.read_excel(datafile1, usecols = "A:E", header = 0) 
options_data["ret"] = np.log(options_data['sp_500_index']) - np.log(options_data['sp_500_index'].shift(1))
T = len(options_data)

In [27]:
T

502

In [16]:
def BScholes(S,K,maturity, r, sigma, delta): 
    d1 = (np.log(S/K) + (r + sigma**2/2)*maturity)/(sigma*np.sqrt(maturity))
    d2 = d1 - sigma*np.sqrt(maturity)
    Nd1 = stats.norm.cdf(d1)
    Nd2 = stats.norm.cdf(d2)
    Nd1neg = stats.norm.cdf(-d1)
    Nd2neg = stats.norm.cdf(-d2)

    callprice = S*np.exp(-delta*maturity)*Nd1 - np.exp(-r*maturity)*K*Nd2
    putprice = np.exp(-r*maturity)*K*Nd2neg - S*np.exp(-delta*maturity)*Nd1neg
    return callprice, putprice

In [20]:
options_data_100=options_data[-100:]

In [ ]:
options_data_100

In [25]:
np.sqrt(252)*options_data_100['ret'].std()   

0.24931501773180081

In [29]:
sigma =  np.sqrt(252)*options_data_100['ret'].std()                              #calculate the annualized vol using the ret from options_data
S =    3901                                 # Fill these in                        
K =   4000
maturity = 50/252
r = 0.04
delta = 0.016
callprice = BScholes(S,K,maturity, r, sigma, delta)[0]
putprice = BScholes(S,K,maturity, r, sigma, delta)[1]
print(f'The call price is {callprice :.2f}.')
print(f'The put price is {putprice :.2f}.')

The call price is 136.91.
The put price is 216.66.


### Q 2.2

In [30]:
options_data_200=options_data[-200:]

In [31]:
sigma200 =  np.sqrt(252)*options_data_200['ret'].std()                                    #calculate the annualized vol using 200 days
callprice200 = BScholes(S,K,maturity, r, sigma200, delta)[0]
putprice200 = BScholes(S,K,maturity, r, sigma200, delta)[1]

print(f'The call price with 200 days vol is {callprice200 :.2f}.')
print(f'The put price with 200 days vol is {putprice200 :.2f}.')


The call price with 200 days vol is 136.02.
The put price with 200 days vol is 215.76.


### Q 2.3

The value of the Vix on 10/28/2022 was xxx%. (Use close)

In [32]:
print(f'The 200 days vol is {sigma200*100 :.2f} percent.')
print(f'The 100 days vol is {sigma*100 :.2f} percent.')

The 200 days vol is 24.80 percent.
The 100 days vol is 24.93 percent.


### Q 2.4

In [36]:
call = 129
put = 185
def callfunc(x):                                      # callfunc takes input the volatility x and ouptputs the call price
    return BScholes(S,K,maturity, r, x, delta)[0]

def putfunc(x):
    return  BScholes(S,K,maturity, r, x, delta)[1]                                   # Fill in the output for the putfunc

impliedvol_call = (scp.optimize.minimize(lambda x: (callfunc(x) - call)**2, x0 = 0.2).x)[0]
impliedvol_put =  (scp.optimize.minimize(lambda x: (put-putfunc(x))**2, x0 = 0.2).x)[0]                           # Perform the optimization 

print(f'The implied vol with call is {impliedvol_call*100 :.2f} percent.')
print(f'The implied vol with put is {impliedvol_put*100 :.2f} percent.')


The implied vol with call is 23.78 percent.
The implied vol with put is 20.30 percent.
